### 卷积网络 Basic
- 卷积网络
- 互相关运算
- 卷积核/过滤器, 不一定是正方形
- 卷积窗口
- 训练: 迭代卷积核和偏差
- 如何通过数据学习卷积核:
    - 使用gluon自带的Conv2D
        - 参数: 
            - 通道数
            - 核形状
        - 二维卷积层使用4维输入输出，格式为(样本, 通道, 高, 宽): conv2d((samples, channels, height, width))
    - 初始化成数组
    - 计算 损失函数的梯度来更新权重和偏差
- 特征图, feature map: 二维卷积层 的输出 可以看做是输入在空间维度(宽和高)上的某一级的表征
- 感受野, receptive field: 影响 元素x的前向计算的所有可能输入区域(可能会超出输入区域) -> 二维的八个方位
    - 可以通过更深的卷积神经网络使特征图中单个元素的感受野变得更加广阔，从而捕捉输入上更大尺寸的特征
    - 比如 X_3*3 -(K1_2*2)-> Y_2*2 -(K2_2*2)-> Z_1*1,  Z在Y上的感受野是Y的全部4个元素, 而在输入(X)上的感受野是输入的全部9个元素
- 怎么设计卷积核, 令其能够检测出边缘?
- dropout(丢弃法):
    - 操作:
        - 随机选择该层一部分的输出作为丢弃元素;
        - 把丢弃元素乘以0;
        - 把非丢弃元素拉伸:
    - 怎么拉伸 ? :
        - scale = 1/keep_prob
        - e.g: dropout prob = 0.5, 
    - 超参: 
        - dropout/keep prob: 丢弃的概率, 为1, 全是0
        - 
    - why dopout: 
        - 随机隐藏模型的一部分, 降低模型复杂度, <b>避免 "过拟合"</b>
        - 差不多数量的参数, 但比纯粹的小模型好
    - when: 
        - 在全连接层输出的激活函数后
        - dropout prob 前面小, 后面大
- 多通道输入和多通道输出
    - what is it ?
        - channel, 通道, e.g: 彩图的RGB三通道
        - 
    - why design it ?
        - 处理多维度的数据
    - when to use it ?
    - how to use it ?
        - 多通道输入 -> 多通道卷积核 -> 按通道相加(使用`nd.add_n(*channels)`) -> 只剩一个输出通道
        - 多通道输入 -> 多通道卷积核 -> 全部输入通道 同时与 各个卷积核 卷积并求和 -> 有几个卷积核, 就有几个输出通道
           - 每个输出通道上的结果由卷积核在该输出通道上的核数组与整个输入数组计算而来
- 1\*1 多通道卷积层
    - 输入层维度 和 输出层一样, 但是 输出层通道数 等于 卷积核通道数
    - 有全连接层的功效
- 池化层(Pooling)
    - what is it ?
        - 池化窗口: 池化层每次对输入数据的一个固定形状窗口中的元素计算输出
        - 直接计算奢池化窗口的最大值或者均值: 最大池化, 平均池化
        - 
    - why use it ?
        - 缓解卷积层对位置的过度敏感性
    - 和卷积层有啥关系?
        - 卷积层的输出, 作为池化层的输入
        - 对于 2x2 池化层, 只要卷积识别的模式在高和宽上移动不超过一个pixel, 都可以检测出来
    - how implement it ?
        - `pool2d()`
    - when use it ?
        - after Cov
    - how use it ?
        - `nn.MaxPool2D`
    - why not use it ? Cons
        - ??
- 池化层的填充与步幅: 同卷积核
- `nn.MaxPool2D`:
    - By default, 步幅和池化窗口形状相同
    - But, 指定填充和步幅: MaxPool2D(size, padding=XXX, strides=YYY), size 为一个数, 就是正方形, 为元组, 就是长方形
    - 不需要参数 -> 超参:  pool_size
- 多通道池化:
    - 分别池化: 输出通道数和 输入通道数相等
- About Pooling:
    - 特征提取的误差主要来自:
        - 邻域大小受限造成的估计值方差增大;
        - 卷积层参数误差造成 均值的平移. 
    - 平均池化, 减小第一种误差, 更多保留图像的背景信息;
    - 最大池化, 减小第二种误差, 更多保留纹理信息
    - Stochastic-pooling, 介于两者之间, 通过对像素点 按照数值大小赋予概率, 再按照概率进行亚取样
        - 平均意义上, 和 mean-pooling 接近
        - 局部意义上, 和 max-pooling 接近
- Pooling 的反向传播, 以 2x2的池化窗口为例
    - 原则: 将1个像素的梯度传递给四个像素, 保证传递的loss总和不变
    - 对于 mean-pooling: 将值四等分, 传给前面的2x2的格子中
    - 对于 max-pooling: 把当前值放到池化窗口中数值最大的位置, 而其它位置 填0  -> 池化操作时需要记录 那个位置的数字最大
- 论文:
    [Learning Mid-Level Features For Recognition](https://www.di.ens.fr/willow/pdfs/cvpr10c.pdf)


In [1]:
from mxnet.gluon import nn
from mxnet import autograd, nd

In [37]:
def corr2d(X, K):
    """
    too slow
    X: input
    K: kernel
    """
    h, w = K.shape
    Y = nd.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

In [3]:
X = nd.random.uniform(shape=(100, 100))

In [4]:
k = nd.ones(shape=(2, 2)) * 2

In [5]:
X


[[0.5488135  0.5928446  0.71518934 ... 0.45720482 0.55868983 0.43069857]
 [0.14038694 0.9391278  0.19810149 ... 0.12132858 0.5108276  0.5693113 ]
 [0.89294696 0.43706194 0.8962931  ... 0.01902474 0.9489773  0.03978036]
 ...
 [0.9162014  0.44308117 0.59071505 ... 0.5811341  0.05543189 0.99660516]
 [0.49628252 0.6452971  0.80752385 ... 0.9595277  0.6766825  0.50212663]
 [0.6899407  0.47096756 0.13747388 ... 0.6698337  0.7846096  0.26094997]]
<NDArray 100x100 @cpu(0)>

In [6]:
0.5488135 * 2 + 0.5928446 * 2 + 0.14038694 * 2 + 0.9391278 * 2

4.44234568

In [7]:
%time corr2d(X, k)

CPU times: user 2.49 s, sys: 227 ms, total: 2.72 s
Wall time: 2.3 s



[[4.4423456 4.8905263 5.071892  ... 2.5734859 3.2961016 4.139055 ]
 [4.8190475 4.941169  3.7830637 ... 1.7348094 3.2003164 4.137793 ]
 [6.1226377 6.289296  4.4553595 ... 3.1869245 3.990566  3.8759232]
 ...
 [6.3993626 4.4471655 3.894683  ... 3.9462676 4.67764   4.0472145]
 [5.0017242 4.973234  5.442731  ... 5.556615  4.5455523 4.4616923]
 [4.6049757 4.1225247 3.9016984 ... 6.3918395 6.181307  4.448737 ]]
<NDArray 99x99 @cpu(0)>

In [8]:
class Conv2D(nn.Block):
    """
    二维卷积层
    """
    def __init__(self, kernel_shape, **kwargs):
        """
        :param kernel_shape: tuple
        """
        super(Conv2D, self).__init__(**kwargs)
        self.weight = self.params.get("weight", shape=kernel_shape)
        self.bias = self.params.get("bias", shape=(1, ))
    
    def forward(self, X):
        return corr2d(X, self.weight.data()) + self.bias.data()

In [9]:
conv2d = Conv2D((3,3))
conv2d.initialize()
conv2d(X)


[[-0.12689734 -0.17974442 -0.13745227 ... -0.12988138 -0.0686918
  -0.15995823]
 [-0.19202366 -0.14812845 -0.11834335 ... -0.14048275 -0.10204579
  -0.12251557]
 [-0.18664989 -0.20418721 -0.1458373  ... -0.13358465 -0.1405009
  -0.1326626 ]
 ...
 [-0.15792166 -0.13353437 -0.15175901 ... -0.11501221 -0.17768392
  -0.13468423]
 [-0.18094483 -0.14080253 -0.16509765 ... -0.11307762 -0.15254816
  -0.1463292 ]
 [-0.15470123 -0.16307648 -0.14882937 ... -0.15462798 -0.19072512
  -0.16540173]]
<NDArray 98x98 @cpu(0)>

In [10]:
conv2d.weight.data()


[[-0.00613896 -0.03968295  0.00958075]
 [-0.05106945 -0.06736943 -0.02462026]
 [ 0.01646897 -0.04904552  0.0156934 ]]
<NDArray 3x3 @cpu(0)>

In [11]:
conv2d.bias.data()


[-0.03887501]
<NDArray 1 @cpu(0)>

### 检测图片中物体的边缘

In [ ]:
# images = nd.ones((6, 8))
images[:, 2:6] = 0
images
#从白到黑, 又从黑到白, 两条分界线

In [13]:
K = nd.array([[1, -1]])
K


[[ 1. -1.]]
<NDArray 1x2 @cpu(0)>

In [14]:
images_edge = corr2d(images, K)

In [15]:
images_edge
# 检测出 黑到白的分界线: 1
# 检测出 白到黑的分极限: -1


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [16]:
def get_net():
    conv2d = nn.Conv2D(1, kernel_size=(1, 2))
    conv2d.initialize()       
    return conv2d

In [17]:
# 二维卷积层使用4维输入输出，格式为(样本, 通道, 高, 宽)，这里批量大小（批量中的样本数）和通道数均为1
def train_cnn(net, features, labels):
    for i in range(10):
        with autograd.record():
            Y_hat = net(features)
            l = (Y_hat - labels) ** 2
        l.backward()
        # 简单起见，这里忽略了偏差
        # conv2d.weight.data()[:] -= 3e-2 * conv2d.weight.grad()
        net.weight.set_data(net.weight.data() - 3e-2 * net.weight.grad())
        if (i + 1) % 2 == 0:
            print('batch %d, loss %.3f' % (i + 1, l.sum().asscalar()))

In [18]:
net = get_net()
train_cnn(net, features=images.reshape((1, 1, 6, 8)), labels=images_edge.reshape((1, 1, 6, 7)))

batch 2, loss 4.758
batch 4, loss 0.798
batch 6, loss 0.134
batch 8, loss 0.022
batch 10, loss 0.004


In [19]:
images


[[1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1.]]
<NDArray 6x8 @cpu(0)>

In [20]:
images_edge


[[ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]
 [ 0.  1.  0.  0.  0. -1.  0.]]
<NDArray 6x7 @cpu(0)>

In [21]:
net.weight.set_data

<bound method Parameter.set_data of Parameter conv0_weight (shape=(1, 1, 1, 2), dtype=<class 'numpy.float32'>)>

In [22]:
net.bias.data()


[0.]
<NDArray 1 @cpu(0)>

In [23]:
def corr2d_multi_in(X, K):
    """
    首先沿着X和K的第0维（通道维）遍历。然后使用*将结果列表变成add_n函数的位置参数
    """
    return nd.add_n(*[corr2d(x, k) for x, k in zip(X, K)])

In [24]:
X = nd.array([
    [[0, 1, 2], [3, 4, 5], [6, 7, 8]],
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
])
K = nd.array([[
    [0, 1], [2, 3]], 
    [[1, 2], [3, 4]]
])

In [25]:
corr2d_multi_in(X, K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

In [38]:
def corr2d_multi_in_out(X, K):
    """
    对K的第0维遍历，每次同输入X做互相关计算。所有结果使用stack函数合并在一起
    """
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [55]:
nd.stack?  # 拼接矩阵

In [35]:
stack_K = nd.stack(K, K + 1, K + 2)
stack_K.shape

(3, 2, 2, 2)

In [40]:
stack_K


[[[[0. 1.]
   [2. 3.]]

  [[1. 2.]
   [3. 4.]]]


 [[[1. 2.]
   [3. 4.]]

  [[2. 3.]
   [4. 5.]]]


 [[[2. 3.]
   [4. 5.]]

  [[3. 4.]
   [5. 6.]]]]
<NDArray 3x2x2x2 @cpu(0)>

In [39]:
corr2d_multi_in_out(X, stack_K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

In [42]:
def corr2d_multi_in_out_1x1(X, K):
    """
    将通道维当作特征维，将高和宽维度上的元素当成数据样本
    """
    c_i, h, k = X.shape
    c_o  = K.shape[0]
    X = X.reshape((c_i, h*k))
    K = K.reshape((c_o, c_i))  # 将通道为当做特征维
    Y = nd.dot(K, X)
    return Y.reshape((c_o, h, k))  # 各通道的维度同 输入

In [43]:
X = nd.random.uniform(shape=(3, 3, 3))
K = nd.random.uniform(shape=(2, 
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
# 做 1×1 卷积时，corr2d_multi_in_out_1x1 与 corr2d_multi_in_out等价
(Y1 - Y2).norm().asscalar() < 1e-6

True

In [52]:
Y1


[[[0.8203548  1.3518901  1.6934745 ]
  [1.5272851  1.2223532  1.363395  ]
  [0.9929103  0.8171209  1.2170742 ]]

 [[0.18804353 0.28822097 0.76854384]
  [0.6081356  0.5390201  0.56824064]
  [0.2752658  0.23744187 0.39660156]]]
<NDArray 2x3x3 @cpu(0)>

In [53]:
Y2


[[[0.82035476 1.3518901  1.6934745 ]
  [1.5272851  1.2223532  1.363395  ]
  [0.9929104  0.81712097 1.2170742 ]]

 [[0.18804353 0.28822097 0.76854384]
  [0.6081356  0.53902006 0.56824064]
  [0.2752658  0.23744187 0.39660156]]]
<NDArray 2x3x3 @cpu(0)>

In [60]:
def pool2d(X, pool_size, mode='max'):
    """
    二维池化层
    :param pool_size: tuple
    """
    x_h, x_w = X.shape
    p_h, p_w = pool_size
    Y = nd.zeros((x_h - p_h + 1, x_w - p_w + 1))
    y_h, y_w = Y.shape
    # 计算输出
    for i in range(y_h):
        for j in range(y_w):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

In [61]:
X = nd.array([
    [0, 1, 2], 
    [3, 4, 5], 
    [6, 7, 8]
])

In [62]:
pool2d(X, (2, 2))


[[4. 5.]
 [7. 8.]]
<NDArray 2x2 @cpu(0)>

In [63]:
pool2d(X, (2, 2), 'avg')


[[2. 3.]
 [5. 6.]]
<NDArray 2x2 @cpu(0)>

In [65]:
X = nd.arange(16).reshape((1, 1, 4, 4))  # 构造 (1, 1, 4, 4)的矩阵

In [66]:
X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]]]
<NDArray 1x1x4x4 @cpu(0)>

In [67]:
pool_square = nn.MaxPool2D(3)

In [68]:
pool_square(X)


[[[[10.]]]]
<NDArray 1x1x1x1 @cpu(0)>

In [69]:
pool_move_fast = nn.MaxPool2D(3, padding=1, strides=2)

In [70]:
pool_move_fast(X)


[[[[ 5.  7.]
   [13. 15.]]]]
<NDArray 1x1x2x2 @cpu(0)>

In [71]:
pool_rectangle = nn.MaxPool2D((2, 3), padding=(1, 2), strides=(2, 3))  # 上下填充2层, 左右填充1层, 形状: 6x6

In [72]:
pool_rectangle(X)


[[[[ 0.  3.]
   [ 8. 11.]
   [12. 15.]]]]
<NDArray 1x1x3x2 @cpu(0)>

In [73]:
concat_X = nd.concat(X, X + 1, dim=1)

In [75]:
concat_X


[[[[ 0.  1.  2.  3.]
   [ 4.  5.  6.  7.]
   [ 8.  9. 10. 11.]
   [12. 13. 14. 15.]]

  [[ 1.  2.  3.  4.]
   [ 5.  6.  7.  8.]
   [ 9. 10. 11. 12.]
   [13. 14. 15. 16.]]]]
<NDArray 1x2x4x4 @cpu(0)>

### 练习
1. 构造一个输入图像X，令它有水平方向的边缘。如何设计卷积核K来检测图像中水平边缘? 如果是对角方向的边缘呢?
2. 试着对我们自己构造的Conv2D类进行自动求梯度，会有什么样的错误信息？在该类的forward函数里，将corr2d函数替换成nd.Convolution类使得自动求梯度变得可行。
    - 对一个for循环如何求梯度
3. 如何通过变化输入和核数组将互相关运算表示成一个矩阵乘法？
4. 如何构造一个全连接层来进行物体边缘检测？
5. 假设输入形状为 ci×h×w ，且使用形状为 co×ci×kh×kw 、填充为 (ph,pw) 、步幅为 (sh,sw) 的卷积核。那么这个卷积层的前向计算分别需要多少次乘法和加法？
    - for each co: 对于每一输出通道
        - for each ci:   对于每一输入通道
            - 两个二维数组 卷积: h*w COV kh*kw:
                - 向下移动 (h - kh + ph + sh) / sh
                - 向右移动 (w - kw + pw + sw) / sw
                - 所以: 乘法次数: Time_Down * Time_Right * kw * kh
                - 加法次数:  Time_Down * Time_Right
        - 所有ci相加     
6. 翻倍输入通道数 ci 和输出通道数 co 会增加多少倍计算？翻倍填充呢？
7. 如果卷积核的高和宽 kh=kw=1 ，能减少多少计算？
8. 最后一个例子中的变量Y1和Y2完全一致吗？原因是什么？
    - 
9. 当卷积窗口不为 1×1 时，如何用矩阵乘法实现卷积计算？
10. 最大池化层和 平均池化层 在 作用上有啥区别?
    - <b>Pooling(池化)的结果是保持某种不变形, e.g: 旋转, 平移, 伸缩etc</b>
    - 最大池化层, 增强图片亮度; 
    - 平均池化层, 减少冲击失真, 模糊, 平滑; 
11. 最小池化层 有没有意义?
    - 最小化是不是把卷积的输出都变0 (似乎不是)
12. 分析池化层的计算复杂度。假设输入形状为 c×h×w ，我们使用形状为 ph×pw 的池化窗口，而且使用 (ph,pw) 填充和 (sh,sw) 步幅。这个池化层的前向计算复杂度有多大？